In [1]:
import numpy as np
import numpy.ma as ma
import os
import string
import sys
import xarray as xr
import netCDF4 as nc4
import scipy.io as io
import glob
import matplotlib.pyplot as plt
import pickle
import itertools
from scipy.signal import butter, lfilter, filtfilt
import warnings
warnings.filterwarnings("ignore")
base = os.getcwd()+"/"
print(base)

/opt/anaconda/envs/geog0111/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda/envs/geog0111/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


/home/ucfaccb/Documents/local_repos/PMIP4-midHolocene/pseudocorals/


# Files

In [2]:
def get_model_names(filenames, model_list):

    '''
    Gets a list of available models (midHolocene & piControl) from the 
    curated_ESGF replica directory. 

    INPUTS:
        - filenames: list, a file glob of the available sos & tos files
        - model_list: list, an empty list to put the model names in

    RETURNS:
        - Nothing

    '''

    for path in filenames:
        model_name = path.split('/')[4]
        model_list.append(model_name)   

In [3]:
sos_filenames_mh = glob.glob('/data/CMIP/curated_ESGF_replica/*/midHolocene/sos*.nc')
sos_filenames_ctrl = glob.glob('/data/CMIP/curated_ESGF_replica/*/piControl/sos*.nc')
sos_filenames_hi = glob.glob('/data/CMIP/curated_ESGF_replica/*/historical/sos*.nc')
tos_filenames_mh = glob.glob('/data/CMIP/curated_ESGF_replica/*/midHolocene/tos*.nc')
tos_filenames_ctrl = glob.glob('/data/CMIP/curated_ESGF_replica/*/piControl/tos*.nc')  
tos_filenames_hi = glob.glob('/data/CMIP/curated_ESGF_replica/*/historical/tos*.nc')

sos_models_mh=[]
sos_models_ctrl=[]
sos_models_hi=[]
tos_models_mh=[]
tos_models_ctrl=[]
tos_models_hi=[]

get_model_names(sos_filenames_mh, sos_models_mh)
get_model_names(sos_filenames_ctrl, sos_models_ctrl)
get_model_names(sos_filenames_hi, sos_models_hi)
get_model_names(tos_filenames_mh, tos_models_mh)
get_model_names(tos_filenames_ctrl, tos_models_ctrl)  
get_model_names(tos_filenames_hi, tos_models_hi)

In [4]:
def get_filenames(sos_models, tos_models, expt, var, array_list, model_namelist):
    
    '''
    Opens up the sos and tos files into x-array datasets.
    
    INPUTS:
        - sos_models, tos_models: list of model names
        - expt: string, midHolocene or piControl
        - var: string, sos or tos
        - array_list: list to put x_array dataset in
        - model_namelist: empty list, to put new model names in
    
    RETURNS:
        - Nothing
    
    '''
    
    # are both sos and tos present in piControl/midHolocene folders?
    for model in sos_models:
        if model in tos_models:
                                                # {model}{expt}{var}
            fn_format= "/data/CMIP/curated_ESGF_replica/{}/{}/{}*.nc"
            # make a file-glob by putting the model into format
            files = fn_format.format(model, expt, var)
            print(files)

            # open datasets & put them in a list
            for fname in glob.iglob(files):
                array_list.append(xr.open_dataset(fname))
                
            model_namelist.append(model)
    
    print('\n')

In [5]:
#-------------------------------------------------------------------------------
new_sos_models_mh=[]
new_sos_models_ctrl=[]
new_sos_models_hi=[]
new_tos_models_mh=[]
new_tos_models_ctrl=[]
new_tos_models_hi=[]

sos_data_mh = []
sos_data_ctrl = []
sos_data_hi = []
tos_data_mh = []
tos_data_ctrl = []
tos_data_hi = []

get_filenames(sos_models_mh, tos_models_mh, 'midHolocene', 'sos', sos_data_mh, 
              new_sos_models_mh)
get_filenames(sos_models_mh, tos_models_mh, 'midHolocene', 'tos', tos_data_mh, 
              new_tos_models_mh)
get_filenames(sos_models_ctrl, tos_models_ctrl, 'piControl', 'sos', sos_data_ctrl, 
              new_sos_models_ctrl)
get_filenames(sos_models_ctrl, tos_models_ctrl, 'piControl', 'tos', tos_data_ctrl, 
              new_tos_models_ctrl)
get_filenames(sos_models_hi, tos_models_hi, 'historical', 'sos', sos_data_hi, 
              new_sos_models_hi)
get_filenames(sos_models_hi, tos_models_hi, 'historical', 'tos', tos_data_hi, 
              new_tos_models_hi)

/data/CMIP/curated_ESGF_replica/CESM2/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/GISS-E2-1-G/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/HadGEM3-GC31/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/IPSL-CM6A-LR/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/NESM3/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/MRI-ESM2-0/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/MIROC-ES2L/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/NorESM1-F/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/UofT-CCSM-4/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/FGOALS-f3-L/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/INM-CM4-8/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/FGOALS-g3/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/NorESM2-LM/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/EC-Earth3-LR/midHolocene/sos*.nc
/data/CMIP/curated_ESGF_replica/MPI-ESM1-2-LR/midHolocene/sos*.nc


/data/CMIP/curated_ESGF_replica/CESM2/midHolocene/tos*

# Coordinate system

In [6]:
#-------------------------------------------------------------------------------
def get_coord_names(fx):
    
    '''
    Discovers what the lat/lon variable in a dataset is called and returns this
    as an array. Also converts -180-->180 deg longitudes to 0-->360 deg
    longitudes.
    
    INPUTS:
        - fx: xarray DataSet e.g. sos_data_ctrl[0]

    RETURNS:
        - lat: array of latitudes
        - lon: array of longitudes 
    
    '''
    
    # work out what lat/lon var is called
    if 'lat' in fx.variables:
        lat = fx.variables['lat'].values
        lon = fx.variables['lon'].values
        if lon.max() < 300: # convert -180-->180 lons to 0-->360 lons
            lon %= 360

    elif 'nav_lat' in fx.variables:
        lat = fx.variables['nav_lat'].values
        lon = fx.variables['nav_lon'].values
        if lon.max() < 300:
            lon %= 360

    elif 'latitude' in fx.variables:
        lat = fx.variables['latitude'].values
        lon = fx.variables['longitude'].values
        if lon.max() < 300:
            lon %= 360
     
    else:
        print("!!LAT/LON VAR NAME NOT RECOGNISED!!")
        
    return lat, lon

In [7]:
#-------------------------------------------------------------------------------
def get_curvi_coords(fx, var, min_lat, max_lat, min_lon, max_lon, verbose):
    
    '''
    This code was developed by Damian Oyarzun Valenzuela, PhD candidate, 
    Geography UCL, 2017.
    
    Returns variable over a specific lat-lon region by taking a subset
    of the curvilinear coords i.e. for a variable X:
        latitude.shape = (y.y)
        longitude.shape = (x.x)
    
    INPUTS:
        - fx: xarray DataSet e.g. sos_data_ctrl[0]
        - var: xarray DataArray e.g. sos_data_ctrl[0].sos
        - min_lat: the minimum latitude (deg N)
        - max_lat: the maximum latitude (deg N)
        - min_lon: the minimum longitude (deg E)
        - max_lon: the maximum longitude (deg E)
        - verbose: if True, calculate the variable (e.g. sos) over the AOI,
                   if False, calculate the lat/lon variable over the AOI 
                   (curvilinear coords)
    
    RETURNS:
        - var_ai: var in a specific lat-lon region
    
    '''
    
    print('***getting curvi coordinates***')
    
    area = [min_lat, max_lat, min_lon, max_lon]  
    lat, lon = get_coord_names(fx)

    # Specify area of interest as lat-lon degrees
    # Produces boolean array 
    latt = np.logical_and(lat >= area[0], lat <= area[1])
    lonn = np.logical_and(lon >= area[2], lon <= area[3])

    # Select area of interest from combination of lat-lon arrays
    # Produces boolean array
    a_int = latt * lonn 

    # Indices for area of interest
    # nonzero returns indices of elements that are non-zero (True)
    (ii,jj) = a_int.nonzero()
    
    if verbose:
        # Var over AOI
        # shape change: e.g. (8400, 384, 320) --> (8400, 185, 239)
        var_ai = var[:, ii.min():ii.max(),jj.min():jj.max()] \
                     *a_int[ii.min():ii.max(),jj.min():jj.max()]
        
        '''        
        # Show lat/lon field
        # Boolean array, var*AOI, var over AOI only  
        vvv = [a_int, var[0,:,:]*a_int, var_ai[0,:,:]]

        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 2))
        for vv,ax in zip(vvv,axes.flat):
            im = ax.imshow(vv, origin=(20,10)) 
        '''
            
    else:
        # Coords over AOI
        # shape change: e.g. (384, 320) --> (185, 239)
        var_ai = var[ii.min():ii.max(),jj.min():jj.max()] \
                     *a_int[ii.min():ii.max(),jj.min():jj.max()]
        
        '''       
        # Show lat/lon field
        # Boolean array, var*AOI, var over AOI only  
        vvv = [a_int, var[:,:]*a_int, var_ai[:,:]]

        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 2))
        for vv,ax in zip(vvv,axes.flat):
            im = ax.imshow(vv, origin=(20,10))
        '''
    
    print(var.shape, '-->', var_ai.shape)
    print('***finished curvi coordinates***')
    
    return var_ai

In [8]:
#-------------------------------------------------------------------------------
def get_pacific_coords(lat, lon, fs, ft, start_lat, end_lat, start_lon, end_lon):
    
    '''
    Determines whether lat/lon variable is rectilinear or curvilinear. If the
    former, it selects a subset of the variable according to a start_lat,
    end_lat, start_lon and end_lon. It then takes the subsequent lat/lon coord 
    as the lat/lon object. If the latter, it works out a subset of the variable 
    and lat/lon variable using the get_curvi_coords function. To gain a subset of
    the lat/lon coordinates, it works out what the lat/lon variable is called
    in the file.
     
    INPUTS:
        - lat: array of latitudes, e.g. fx.variables['lat'].values
        - lon: array of longitudes, e.g. fx.variables['lon'].values
        - start_lat: the minimum latitude (deg N)
        - end_lat: the maximum latitude (deg N)
        - start_lon: the minimum longitude (deg E)
        - end_lon: the maximum longitude (deg E)    
        - fs, ft: xarray DataSet e.g. tos_data_ctrl[0]
    
    RETURNS:
        - IPsosVar, IPtosVar: var in a specific lat-lon region
        - sos_latobj: latitude in a specific lat-lon region
        - sos_lonobj: longitude in a specific lat-lon region
    
    '''

    # rectilinear
    #try:
    if len(lat.shape) == 1:
        IPsosVar = fs.sos.sel(lat=slice(start_lat, end_lat), 
                              lon=slice(start_lon, end_lon))
        IPtosVar = ft.tos.sel(lat=slice(start_lat, end_lat), 
                              lon=slice(start_lon, end_lon))
        sos_latobj = IPsosVar.lat
        sos_lonobj = IPsosVar.lon

    # curvilinear
    else:
        IPsosVar = get_curvi_coords(fs, fs.sos, start_lat, end_lat, 
                                    start_lon, end_lon, verbose=True)
        IPtosVar = get_curvi_coords(ft, ft.tos, start_lat, end_lat, 
                                    start_lon, end_lon, verbose=True)
        
        if 'lat' in fs.variables:
            sos_latobj = get_curvi_coords(fs, fs.lat, start_lat, end_lat, 
                                          start_lon, end_lon, verbose=False)
            sos_lonobj = get_curvi_coords(ft, ft.lon, start_lat, end_lat, 
                                          start_lon, end_lon, verbose=False)

        elif 'nav_lat' in fs.variables:
            sos_latobj = get_curvi_coords(fs, fs.nav_lat, start_lat, end_lat, 
                                          start_lon, end_lon, verbose=False)
            sos_lonobj = get_curvi_coords(ft, ft.nav_lon, start_lat, end_lat, 
                                          start_lon, end_lon, verbose=False)

        elif 'latitude' in fs.variables:
            sos_latobj = get_curvi_coords(fs, fs.latitude, start_lat, end_lat, 
                                          start_lon, end_lon, verbose=False)
            sos_lonobj = get_curvi_coords(ft, ft.longitude, start_lat, end_lat, 
                                          start_lon, end_lon, verbose=False)
    
    return IPsosVar, IPtosVar, sos_latobj, sos_lonobj

# Compute corals

In [9]:
#-------------------------------------------------------------------------------
def coral_sensor_field(latArray, lonArray, sst, sss):
    
    '''  
    This function implements the bivariate model of [1] to SST and SSS fields.
    Adapted from: https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/pmip3_pcoral.py
    
    INPUTS:  
        - latArray, lonArray, numpy 1D arrays
        - sst (in K or degC), masked array
        - sss (in psu*), masked array
        
    RETURNS
        - coral, the pseudocoral at the same locations as SST, SSS
        - tosContri, the thermal contribution
        - sosContri, the hydrological contribution

    * assumes SSS in psu, so need to convert if this is not the case 
      
    [1] Thompson, D. M. , T. R. Ault , M. N. Evans , J. E. Cole , 
    and J. Emile-Geay (2011), Comparison of observed and simulated tropical 
    climate trends using a forward model of coral δ18O, Geophys. Res. 
    Lett., 38, L14706, doi:10.1029/2011GL048224.  
    
    '''
    
    print('***doing coral_sensor_field***')
    print('centering the fields')
    # center the fields
    nt, ny, nx = sss.shape
    # Mean over time
    sss_m = ma.mean(sss, axis=0)
    sss_c = sss - np.tile(sss_m, (nt,1,1)) 
    sst_m = ma.mean(sst, axis=0)
    sst_c = sst - np.tile(sst_m, (nt,1,1)) 
    
    print('assigning b-values')
    # assign different b values based on location
    a = -0.22
    b1 = 0.3007062
    b2 = 0.1552032
    b3 = 0.2619054
    b4 = 0.436509
    
    b = np.empty((len(latArray),len(lonArray)))
    for lat in range(len(latArray)):
        for lon in range(len(lonArray)):
            #Red sea
            if lonArray[lon]>=32.83 and lonArray[lon]<=43.5 and \
            latArray[lat]>=12.38 and latArray[lat]<=28.5:
                b[lat][lon]=b1
            #Indian ocean
            elif lonArray[lon]<=120:
                b[lat][lon]=b2
            #Tropical Pacific
            elif latArray[lat]>= -5 and latArray[lat]<=13:
                b[lat][lon]=b3
            #South Pacific
            elif latArray[lat]< -5:
                b[lat][lon]=b4
            #Default: Tropical Pacific
            else:
                b[lat][lon]=b3

    print('storing b-values')
    # store coordinates of four b values seperately
    b1_index = np.where(b == b1)
    b2_index = np.where(b == b2)
    b3_index = np.where(b == b3)
    b4_index = np.where(b == b4)

    # create a new array with the same shape as IPsos and compute coral
    coral = np.empty_like(sss)
    tosContri = np.empty_like(sst)
    sosContri = np.empty_like(sss)
    
    print('calculating contributions')
    # hydrological contribution
    for b_index, b in ((b1_index, b1), (b2_index, b2), 
                       (b3_index, b3), (b4_index, b4)):
        sosContri[:, b_index[0], b_index[1]] = b * sss_c[:, b_index[0], 
                                                         b_index[1]]
        
    # thermal contribution    
    tosContri = a * sst_c
    
    # total contribution
    coral = sosContri + tosContri
    
    print('***finished coral_sensor_field***')
    
    # export all three
    return coral, tosContri, sosContri

In [10]:
#-------------------------------------------------------------------------------
def compute_corals(IPsosVar, IPtosVar, sos_latobj, sos_lonobj):
    
    '''  
    This function implements the bivariate model of [1] to SST and SSS fields.
    Adapted from: https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/pmip3_pcoral.py
    
    INPUTS:  
        - IPsosVar, IPtosVar: var in a specific lat-lon region
        - sos_latobj: latitude in a specific lat-lon region
        - sos_lonobj: longitude in a specific lat-lon region
        
    RETURNS
        - tobj: the time variable
        - sos_latobj: latitude in a specific lat-lon region
        - sos_lonobj: longitude in a specific lat-lon region
        - coral2: the pseudocoral at the same locations as SST, SSS
        - tosContri: the thermal contribution
        - sosContri: the hydrological contribution
        
    * assumes SSS in psu, so need to convert if this is not the case 
      
    [1] Thompson, D. M. , T. R. Ault , M. N. Evans , J. E. Cole , 
    and J. Emile-Geay (2011), Comparison of observed and simulated tropical 
    climate trends using a forward model of coral δ18O, Geophys. Res. 
    Lett., 38, L14706, doi:10.1029/2011GL048224.  
    
    '''
    
    print('***starting compute_corals***')

    # define missing values
    ma.set_fill_value(IPsosVar, 1e20)
    ma.set_fill_value(IPtosVar, 1e20)

    # load into arrays
    IPsos = IPsosVar.values
    IPtos = IPtosVar.values

    # get the values for computations
    sos_ma = ma.masked_equal(IPsos, 1e20)
    sos_ma = ma.array(sos_ma, mask=np.isnan(sos_ma))
    tos_ma = ma.masked_equal(IPtos, 1e20)
    tos_ma = ma.array(tos_ma, mask=np.isnan(tos_ma))

    # get the means map
    sos_mean = ma.mean(sos_ma, axis=0)
    tos_mean = ma.mean(tos_ma, axis=0)

    # total means no seasonal cycle is removed from the computation
    sos_mean_total = ma.mean(sos_ma)
    if sos_mean_total <= 1:   #  MORE SOPHISTICATED EXCEPTION HANDLING HERE?
        print ('times sos by 1000')
        sos_ma = sos_ma * 1000

    tobj = IPsosVar.time
    timeArray = tobj.values

    print('creating lat/lon arrays')
    # detect whether variable is in curvilinear grid
    # curvilinear
    if len(sos_latobj.shape) == 2:
        latArray = sos_latobj[:,0]
        lonArray = sos_lonobj[0,:]
    # rectangular
    else:
        latArray = sos_latobj
        lonArray = sos_lonobj

    # apply coral sensor model 
    coral, tosContri, sosContri = coral_sensor_field(latArray, lonArray, 
                                                     tos_ma, sos_ma)

    print('creating masked arrays')
    coral2 = ma.masked_equal(coral, 1e20) # coral2.shape = 1200, 30, 108
    tosContri = ma.masked_equal(tosContri, 1e20)
    sosContri = ma.masked_equal(sosContri, 1e20)
    
    print('***finished compute_corals***')

    return tobj, sos_latobj, sos_lonobj, coral2, tosContri, sosContri

In [11]:
#-------------------------------------------------------------------------------
def coral_sensor_apply(ft, fs, expt, model):
    
    '''
    This function converts model output to pseudocoral, according to the 
    bivariate model of [1]. 
    Adapted from: https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/pmip3_pcoral.py
    
    INPUTS:  
        - ft: filename for SST field, with variable name tname [default = 'tos']
        - fs: filename object field, with variable name sname [default = 'sos']
        
    RETURNS
        - eastern_vars, central_vars, western_vars: tuples of objects --> 
          e_tobj, e_sos_latobj, e_sos_lonobj, e_coral2, e_tosContri, e_sosContri

    [1] Thompson, D. M. , T. R. Ault , M. N. Evans , J. E. Cole , 
    and J. Emile-Geay (2011), Comparison of observed and simulated tropical 
    climate trends using a forward model of coral δ18O, Geophys. Res. Lett., 
    38, L14706, doi:10.1029/2011GL048224.
    
    '''
  
    print('***starting coral_sensor_apply***')
    
    # get the start and end time steps
    start_time_sos = fs.time[0] 
    end_time_sos   = fs.time[-1]
    start_time_tos = ft.time[0]
    end_time_tos   = ft.time[-1]
    
    print('getting variables & lat/lon objects')
    sos_latobj, sos_lonobj = get_coord_names(fs)
    
    # EAST PACIFIC: -10, 0, 270, 280
    e_IPsosVar, e_IPtosVar, e_sos_latobj, e_sos_lonobj = \
    get_pacific_coords(sos_latobj, sos_lonobj, fs, ft, -10, 0, 270, 280) 
    
    # CENTRAL PACIFIC: -5, 5, 190, 240
    c_IPsosVar, c_IPtosVar, c_sos_latobj, c_sos_lonobj = \
    get_pacific_coords(sos_latobj, sos_lonobj, fs, ft, -5, 5, 190, 240) 

    # WEST PACIFIC: -20, 0, 120, 180
    w_IPsosVar, w_IPtosVar, w_sos_latobj, w_sos_lonobj = \
    get_pacific_coords(sos_latobj, sos_lonobj, fs, ft, -20, 0, 120, 180) 
    

    e_tobj, \
    e_sos_latobj, \
    e_sos_lonobj, \
    e_coral2, \
    e_tosContri, \
    e_sosContri = compute_corals(e_IPsosVar,e_IPtosVar,e_sos_latobj,e_sos_lonobj)
    
    c_tobj, \
    c_sos_latobj, \
    c_sos_lonobj, \
    c_coral2, \
    c_tosContri, \
    c_sosContri = compute_corals(c_IPsosVar,c_IPtosVar,c_sos_latobj,c_sos_lonobj)
    
    w_tobj, \
    w_sos_latobj, \
    w_sos_lonobj, \
    w_coral2, \
    w_tosContri, \
    w_sosContri = compute_corals(w_IPsosVar,w_IPtosVar,w_sos_latobj,w_sos_lonobj)
    
    eastern_vars = e_tobj, e_sos_latobj, e_sos_lonobj, e_coral2, e_tosContri, e_sosContri
    central_vars = c_tobj, c_sos_latobj, c_sos_lonobj, c_coral2, c_tosContri, c_sosContri
    western_vars = w_tobj, w_sos_latobj, w_sos_lonobj, w_coral2, w_tosContri, w_sosContri
    
    ###########################
    # create a dictionary to store corals
    corals = {}
    corals['east'] = e_coral2
    corals['central'] = c_coral2
    corals['west'] = w_coral2
    
    # save dictionary to a pickle file
    pickle.dump(corals, open(base + 'corals/coral_{}_{}.p'.format(expt, model), "wb" ))

    # save .mat
    io.savemat(base + 'corals/coral_{}_{}.mat'.format(expt, model), corals)

    print("saved!")
    
    return eastern_vars, central_vars, western_vars

# Bandpass & bootstrapping

In [12]:
def butter_bandpass(lowcut, highcut, fs, order=4):
    '''
    Adapted from https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/bandpass.py
    '''
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a

In [13]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    '''
    Adapted from https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/bandpass.py
    '''
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

The bootstrap method is a statistical technique for estimating quantities about a population by averaging estimates from multiple small data samples. Samples are constructed by drawing observations from a large data sample one at a time and returning them to the data sample after they have been chosen. This allows a given observation to be included in a given small sample more than once. This approach to sampling is called sampling with replacement.

<h6>Moving blocks sampling</h6>
Break the series into roughly equal-length blocks of consecutive observations, to resample the block with replacement, and then paste the blocks together. There are $(n - b + 1)$ such blocks available, with consecutive samples of length b. This preserves dependency in the original samples to length b. 

For each boostrap sample, randomly select blocks and assemble into a length-n timeseries, then compute $\hat{\beta}^*$ for each such length-n series.

In [14]:
def block_bootstrap_ET(X, Lb, Nb): 
    
    '''
    Implement Block Bootstrap as in: 
    http://nbviewer.ipython.org/github/welch/stats-notebooks/
    blob/master/SubsamplingBootstrap.ipynb.
    Adapted from https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/bootstrap.py.
    
    INPUTS:
        - X: the bootstrap sample, array
        - Lb: needed to sample multiples of 12 years
        - Nb: number of bootstrap samples 
    
    RETURNS:
        - Xb: numpy array, resampled version, or "replicate" of data
    '''
    
    nt = len(X)
    ns = int(np.ceil(nt/Lb))
    Xb = np.zeros((Nb, nt))

    for b in range(Nb):       
        for block_i, start in enumerate(np.random.randint(nt - Lb + 1, size=ns)):
            try:
                Xb[b, block_i*Lb:(block_i+1)*Lb] = X[start:start+Lb]
            except ValueError: 
            # changing Lb to 12 as 24 would make (block_i+1)*Lb out of range for X
                Xb[b, block_i*12:(block_i+1)*12] = X[start:start+12]

    return Xb

In [15]:
def seasonal_cycle(Xb):
    
    '''
    Compute and isolate the seasonal cycle.
    Adapted from https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/pcoral_bootstrap.py
    
    INPUTS:
        - Xb: numpy array, resampled version, or "replicate" of data
    
    RETURNS:
        - clim:
        - anom:
    '''
    
    nb,nt = Xb.shape
    ny    = int(nt/12)
    clim  = np.empty((nb,12))

    for i in range(12):
        clim[:,i] = Xb[:,i::12].mean(axis=1)
    print("clim", clim.shape)

    anom = Xb - np.tile(clim,(1, ny))
    
    return clim, anom

In [16]:
def computer(coral, Nb, Lb, windows):
    
    '''
    Compute variance & seasonal amplitude.
    Adapted from https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/pcoral_bootstrap.py
    
    INPUTS:
        - coral:
        - Nb: number of bootstrap samples 
        - Lb: needed to sample multiples of 12 years
        - windows: sampling windows
    
    RETURNS:
        - variance, seasonal_amp:
    '''

    # filtering parameters    
    fs = 1
    f_hi = 1/(12*2.0)
    f_lo = fs/(12*7.0)
    
    # compute spatial mean
    spatial_mean = coral.mean(axis=(1,2))
    print("spatial_mean", spatial_mean.shape)
    print("coral", coral.shape)
    
    # generate boostrap samples
    Xb = block_bootstrap_ET(spatial_mean, Lb, Nb)
    nw = len(windows) # number of windows
   
    seasonal_amp = np.empty((nw, Nb))
    variance = np.empty((nw, Nb))
    
    index = 0  # loop over windows
    for i in windows:
        Xw =  Xb[:, :i*12]  # sample over window
        clim, anom = seasonal_cycle(Xw)  # isolate seasonal cycle
        # compute seasonal amplitude
        smax = np.nanmax(clim, axis=1)
        smin = np.nanmin(clim, axis=1)
        seasonal_amp[index, :] = smax - smin
        
        # compute ENSO variance
        anom2_7 = np.empty(anom.shape)
        for b in range(Nb):
            # apply bandpass filter        
            anom2_7[b, :] = butter_bandpass_filter(anom[b,:], f_lo, f_hi, fs)
        # compute variance per se     
        variance[index,:]  = np.var(anom2_7,axis=1)
        index +=1  # update index
        
    return (variance, seasonal_amp)

In [17]:
#-------------------------------------------------------------------------------
def create_stats(tos_data, sos_data, expt, model):
    
    '''
    Creates and stores statistics.
    Adapted from https://github.com/CommonClimate/EmileGeay_NatGeo2015/blob/
    master/code/python/pmip3_pcoral_bootstrap.py
    
    INPUTS:
        - tos_data
        - sos_data

    
    RETURNS:
        - variance, seasonal_amp:
    '''

    # This script uses block bootstrap to randomize coral data [uses different 
    # sampling time length to generate distribution plot of seasonal cycle amplitude]?
    
    eastern_vars, central_vars, western_vars = coral_sensor_apply(tos_data, 
                                                                  sos_data, 
                                                                  expt, model)

    Nb = 1000 # number of bootstrap samples 
    Lb = 24 # needed to sample multiples of 12 years
    windows = [50] # observation windows
    nw = windows.__len__()

    pcoral_boot_exp = {}; variance = {}; seasonal_amp = {}

    # compute bootstrapped climate statistics on the three AOI
    variance_e, seasonal_amp_e = computer(eastern_vars[3], Nb, Lb, windows)
    variance_c, seasonal_amp_c = computer(central_vars[3], Nb, Lb, windows)
    variance_w, seasonal_amp_w = computer(western_vars[3], Nb, Lb, windows)

    # store variance results
    variance = np.empty((3*nw, Nb))

    variance[0:nw, :] = variance_e
    variance[nw:2*nw, :] = variance_c
    variance[2*nw:3*nw, :] = variance_w

    # store seasonal amplitude results
    seasonal_amp = np.empty((3*nw, Nb))
    seasonal_amp[0:nw, :] = seasonal_amp_e
    seasonal_amp[nw:2*nw, :] = seasonal_amp_c
    seasonal_amp[2*nw:3*nw, :] = seasonal_amp_w

    pcoral_boot_exp['var'] = variance
    pcoral_boot_exp['seas'] = seasonal_amp   

    print(variance.shape, seasonal_amp.shape) # 0:6=east, 6:12=central, 12:18=west
    print("Done!")
    
    ###########################
    
    # save dictionary to a pickle file
    pickle.dump(pcoral_boot_exp, open( base + 'bootstrapped_corals/pcoral_bootstrap_{}_{}.p'. \
               format(expt, model), "wb" ))

    # save .mat
    io.savemat(base + 'bootstrapped_corals/pcoral_bootstrap_{}_{}.mat'. \
               format(expt, model), pcoral_boot_exp)

    print("saved!")

In [18]:
for i, model in enumerate(new_sos_models_mh):
    outfile='corals/coral_mh_MODNAME.mat'
    already_exists=os.path.isfile(outfile.replace('MODNAME',new_sos_models_mh[i]))
    if not already_exists:
        print("DOING MODEL: ",  new_sos_models_mh[i])
        create_stats(tos_data_mh[i], sos_data_mh[i], 'mh', new_sos_models_mh[i])

DOING MODEL:  FGOALS-f3-L
***starting coral_sensor_apply***
getting variables & lat/lon objects
***getting curvi coordinates***
(1200, 218, 360) --> (1200, 17, 10)
***finished curvi coordinates***
***getting curvi coordinates***
(1200, 218, 360) --> (1200, 17, 10)
***finished curvi coordinates***
***getting curvi coordinates***
(218, 360) --> (17, 10)
***finished curvi coordinates***
***getting curvi coordinates***
(218, 360) --> (17, 10)
***finished curvi coordinates***
***getting curvi coordinates***
(1200, 218, 360) --> (1200, 18, 50)
***finished curvi coordinates***
***getting curvi coordinates***
(1200, 218, 360) --> (1200, 18, 50)
***finished curvi coordinates***
***getting curvi coordinates***
(218, 360) --> (18, 50)
***finished curvi coordinates***
***getting curvi coordinates***
(218, 360) --> (18, 50)
***finished curvi coordinates***
***getting curvi coordinates***
(1200, 218, 360) --> (1200, 30, 60)
***finished curvi coordinates***
***getting curvi coordinates***
(1200, 218,

In [19]:
for i, model in enumerate(new_sos_models_ctrl):
    outfile='corals/coral_ctrl_MODNAME.mat'
    already_exists=os.path.isfile(outfile.replace('MODNAME',new_sos_models_ctrl[i]))
    if not already_exists:
        print("DOING MODEL: ",  new_sos_models_ctrl[i])
        create_stats(tos_data_ctrl[i], sos_data_ctrl[i], 'ctrl', new_sos_models_ctrl[i])

In [20]:
for i, model in enumerate(new_sos_models_hi):
    outfile='corals/coral_hi_MODNAME.mat'
    already_exists=os.path.isfile(outfile.replace('MODNAME',new_sos_models_hi[i]))
    if not already_exists:
        print("DOING MODEL: ",  new_sos_models_hi[i])
        create_stats(tos_data_hi[i], sos_data_hi[i], 'hi', new_sos_models_hi[i])